In [ ]:
#https://www.originlab.com/python/doc/originpro/index.html
import originpro as op
import os
import numpy as np
import pandas as pd
%matplotlib widget
from matplotlib.widgets import Cursor
import ipywidgets as widgets
from matplotlib.widgets import RadioButtons

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from numpy import cos as cos
from numpy import sin as sin
from numpy import sqrt as sqrt
from numpy import radians as radians
from numpy import pi as pi

In [ ]:
#Initial Parameters
#Template Path
template = os.path.normpath(r'F:\Mirror\Project\TIFR\GitHub\THz\ImporttoFFT.otwu')
graph_template = os.path.normpath(r'C:\Users\ajink\Documents\OriginLab\User Files\Templates')

datapath = os.path.normpath(r'C:\Users\ajink\Desktop\Kachra\180921')
date = '180921'
run = [6,7,8,9,10,11,12,13,15,16]
total_files = [1,1,1,1,1,1,1,1,1,1]
startpos = 0
start = 0
end   = 50000

In [ ]:
#My Functions
def readfile(filename):
    import pandas as pd
    import numpy as np
    
    df = pd.read_csv(filename,skiprows=12,skipfooter=1,sep='\s+',engine='python').iloc[:-1,:5]
    df=pd.DataFrame(np.array(df, dtype=float))
    index=["Distance (um)", "Delay (ps)", "Signal (V)", "Phase (deg)", "Rcos (theta)"]
    df=df.set_axis(index, axis=1, inplace=False).reset_index(drop=True)
    return df

def run_numbering(i):
    if i<10:
        temp_name = f'00%i'%i
    elif i>=100:
        temp_name = f'%i'%i
    else:
        temp_name = f'0%i'%i
    return temp_name

def filename_generator(date, run, total_files,startpos,string):
    i = 0
    filenames = []
    for run_ in run:
        for total_files_ in range(total_files[i]):
            #filename = date + '_' + str(run_numbering(run_)) + 'forward_proct_' + str(total_files_) + '.txt'
            #filename = date + '_' + str(run_numbering(run_)) + '_br_' + str(total_files_) + '.txt'
            filename = date + '_' + str(run_numbering(run_)) + '_'+string+'_proc_' + str(startpos+total_files_) + '.txt'
            filenames.append(filename)
            #filename = date + '_' + str(run_numbering(run_)) + '_ft_' + str(total_files_) + '.txt'
            #filenames.append(filename)
        i = i+1
    return filenames

#Reference: https://www.techiedelight.com/convert-given-number-corresponding-excel-column-name/
# Function to convert a given number to an Excel column
def getColumnName(n):
 
    # initialize output string as empty
    result = ""
 
    while n > 0:
 
        # find the index of the next letter and concatenate the letter
        # to the solution
 
        # here index 0 corresponds to `A`, and 25 corresponds to `Z`
        index = (n - 1) % 26
        result += chr(index + ord('A'))
        n = (n - 1) // 26
 
    return result[::-1]

def importfiles(filepath):
    template = os.path.normpath(r'F:\Mirror\Project\TIFR\GitHub\THz\ImporttoFFT.otwu')
    
    for filepath_ in filepath:
        #op.new_book(type = _WORKBOOK, lname='', template='',hidden=False)
        print(filepath_)
        newbook = op.new_book('w',filepath_,template,False)

        #Activate the template sheet
        activesheet = op.find_sheet()

        #read the file from where the data is to imported
        df = readfile(os.path.normpath(datapath + '\\' + filepath_))

        #Set Interpolation Formula
        activesheet.from_list(20, df.to_numpy()[:,0].shape[0]+1)
        activesheet.from_list(21, df.to_numpy()[:,0][0])
        activesheet.from_list(22, df.to_numpy()[:,0][-1])

        #Place the data into the worksheet
        activesheet.from_list(0, df.iloc[:,0])
        activesheet.from_list(1, df.iloc[:,1])
        activesheet.from_list(2, df.iloc[:,2])
        activesheet.from_list(3, df.iloc[:,3])
        activesheet.from_list(4, df.iloc[:,4])

        #Wait untill Recalculation is complete
        op.wait('r', 0.0)
        
def importfiles_cut(filepath,n):
    template = os.path.normpath(r'F:\Mirror\Project\TIFR\GitHub\THz\ImporttoFFT.otwu')
    
    for filepath_ in filepath:
        #op.new_book(type = _WORKBOOK, lname='', template='',hidden=False)
        newbook = op.new_book('w',filepath_,template,False)

        #Activate the template sheet
        activesheet = op.find_sheet()

        #read the file from where the data is to imported
        df = readfile(os.path.normpath(datapath + '\\' + filepath_))

        #Set Interpolation Formula
        activesheet.from_list(20, df.to_numpy()[:n,0].shape[0]+1)
        activesheet.from_list(21, df.to_numpy()[:n,0][0])
        activesheet.from_list(22, df.to_numpy()[:n,0][-1])

        #Place the data into the worksheet
        activesheet.from_list(0, df.iloc[:n,0])
        activesheet.from_list(1, df.iloc[:n,1])
        activesheet.from_list(2, df.iloc[:n,2])
        activesheet.from_list(3, df.iloc[:n,3])
        activesheet.from_list(4, df.iloc[:n,4])

        #Wait untill Recalculation is complete
        op.wait('r', 0.0)
        
        
def collector(col_name,sheet,filepath,comment,longname,unit):
    sheet.set_formula('A','[Amplitude]Sheet1!A')
    increment = 2
    for filepath_ in filepath:
        col = getColumnName(increment)
        sheet.set_formula(col,'[' + filepath_ +  ']Sheet1!'+col_name)
        sheet.set_label(col,longname,'L')
        sheet.set_label(col,unit,'U')
        sheet.set_label(col,str(comment[increment-2]),'C')
        increment = increment+1

def collector_trans_phase(sheet,workingsheet,col_in,operation_in):
    sheet.set_formula('A','[Amplitude]Sheet1!A')
    increment = 2
    for filepath_ in filepath:
        col = getColumnName(increment)
        col_name = getColumnName(increment+1)
        sheet.set_formula(col,'['+workingsheet+']Sheet1!'+col_name + operation_in +'['+workingsheet+']Sheet1!'+col_in)
        sheet.set_label(col,'Tranmittance','L')
        #sheet.set_label(col,,'U')
        sheet.set_label(col,comment[increment-2],'C')
        increment = increment+1

def refractiveindex(sheet,filepath, comment):
    sheet.set_formula('A','[Amplitude]Sheet1!A')
    sheet.set_label('A','Frequency','L')
    sheet.set_label('A','THz','U')

    increment = 2
    for filepath_ in filepath:

        col = getColumnName(increment)
        comments = str(comment[increment-2])

        n_ = '1-((radians([Phasediff]Sheet1!' + col + '-[Phasediff]Sheet2!B'+ str(increment) +')*0.04771345)/(Col(A)*[Phasediff]Sheet2!B1))'
        sheet.set_formula(col,n_)
        sheet.set_label(col,'Refractive Index','L')
        sheet.set_label(col,'radians','U')
        sheet.set_label(col,comments,'C')
        op.wait('r', 0.0)
        increment = increment+1

def Sparameters():
    df = pd.read_csv('Stokesparameters.txt',header=None,sep='\t').to_numpy()
    
    S0 = op.new_sheet(lname='S0')
    S1 = op.new_sheet(lname='S1')
    S2 = op.new_sheet(lname='S2')
    S3 = op.new_sheet(lname='S3')
    orientation = op.new_sheet(lname='orientation')
    ellipticity = op.new_sheet(lname='ellipticity')
    
    S0.set_formula('A','[Amplitude]Sheet1!A')
    S1.set_formula('A','[Amplitude]Sheet1!A')
    S2.set_formula('A','[Amplitude]Sheet1!A')
    S3.set_formula('A','[Amplitude]Sheet1!A')
    orientation.set_formula('A','[Amplitude]Sheet1!A')
    ellipticity.set_formula('A','[Amplitude]Sheet1!A')
    
    filename = df[:,0].tolist()
    B = df[:,1].tolist()
    C = df[:,2].tolist()
    D = df[:,3].tolist()
    E = df[:,4].tolist()
    F = df[:,5].tolist()
    G = df[:,6].tolist()

    length = len(B)
    template = os.path.normpath(r'F:\Mirror\Project\TIFR\GitHub\THz\Stokesparam.otwu')
    
    for i in range(length):
        increment = i+2
        
        col = getColumnName(increment)
        filename_ = filename[i]
        
        #op.new_book(type = _WORKBOOK, lname='', template='',hidden=False)
        newbook = op.new_book('w',str(filename_),template,False)

        #Activate the template sheet
        activesheet = op.find_sheet()

        #Place the data into the worksheet
        activesheet.set_formula('A','[Amplitude]Sheet1!A')
        activesheet.set_formula('B','['+ B[i] +']Sheet1!M')
        activesheet.set_formula('C','['+ C[i] +']Sheet1!M')
        activesheet.set_formula('D','['+ D[i] +']Sheet1!M')
        activesheet.set_formula('E','['+ E[i] +']Sheet1!M')
        activesheet.set_formula('F','['+ F[i] +']Sheet1!M')
        activesheet.set_formula('G','['+ G[i] +']Sheet1!M')

        #Wait untill Recalculation is complete
        op.wait('r', 0.0)
        
        S0.set_formula(col,'['+str(filename_)+']Sheet1!R')
        S1.set_formula(col,'['+str(filename_)+']Sheet1!S')
        S2.set_formula(col,'['+str(filename_)+']Sheet1!T')
        S3.set_formula(col,'['+str(filename_)+']Sheet1!U')
        orientation.set_formula(col,'['+str(filename_)+']Sheet1!V')
        ellipticity.set_formula(col,'['+str(filename_)+']Sheet1!W')
        op.wait('r', 0.0)
        
        
def Sparameters2():
    df = pd.read_csv('Stokesparameters.txt',header=None,sep='\t').to_numpy()
    
    S0 = op.new_sheet(lname='S0')
    S1 = op.new_sheet(lname='S1')
    S2 = op.new_sheet(lname='S2')
    S3 = op.new_sheet(lname='S3')
    orientation_s = op.new_sheet(lname='orientation_stokes')
    ellipticity_s = op.new_sheet(lname='ellipticity_stokes')
    
    a = op.new_sheet(lname='a')
    b = op.new_sheet(lname='b')
    orientation_f = op.new_sheet(lname='orientation_formulation')
    ellipticity_f = op.new_sheet(lname='ellipticity_formulation')
    
    S0.set_formula('A','[Amplitude]Sheet1!A')
    S1.set_formula('A','[Amplitude]Sheet1!A')
    S2.set_formula('A','[Amplitude]Sheet1!A')
    S3.set_formula('A','[Amplitude]Sheet1!A')
    orientation_s.set_formula('A','[Amplitude]Sheet1!A')
    ellipticity_s.set_formula('A','[Amplitude]Sheet1!A')
    
    a.set_formula('A','[Amplitude]Sheet1!A')
    b.set_formula('A','[Amplitude]Sheet1!A')    
    orientation_f.set_formula('A','[Amplitude]Sheet1!A')
    ellipticity_f.set_formula('A','[Amplitude]Sheet1!A')

    
    filename = df[:,0].tolist()
    B = df[:,1].tolist()
    C = df[:,2].tolist()
    D = df[:,3].tolist()
    E = df[:,4].tolist()
    F = df[:,5].tolist()
    G = df[:,6].tolist()
    
    Y = df[:,7].tolist()
    Z = df[:,8].tolist()
    AA = df[:,9].tolist()

    length = len(B)
    template = os.path.normpath(r'F:\Mirror\Project\TIFR\GitHub\THz\Stokesparam2.otwu')
    
    for i in range(length):
        increment = i+2
        
        col = getColumnName(increment)
        filename_ = filename[i]
        
        #op.new_book(type = _WORKBOOK, lname='', template='',hidden=False)
        newbook = op.new_book('w',str(filename_),template,False)

        #Activate the template sheet
        activesheet = op.find_sheet()

        #Place the data into the worksheet
        activesheet.set_formula('A','[Amplitude]Sheet1!A')
        activesheet.set_formula('B','['+ B[i] +']Sheet1!M')
        activesheet.set_formula('C','['+ C[i] +']Sheet1!M')
        activesheet.set_formula('D','['+ D[i] +']Sheet1!M')
        activesheet.set_formula('E','['+ E[i] +']Sheet1!M')
        activesheet.set_formula('F','['+ F[i] +']Sheet1!M')
        activesheet.set_formula('G','['+ G[i] +']Sheet1!M')
        
        
        activesheet.set_formula('Y','[Transmittance]Sheet1!'+Y[i])
        activesheet.set_formula('Z','[Transmittance]Sheet1!'+Z[i])
        activesheet.set_formula('AA','[Transmittance]Sheet1!'+AA[i])
        

        #Wait untill Recalculation is complete
        op.wait('r', 0.0)
        
        S0.set_formula(col,'['+str(filename_)+']Sheet1!R')
        S1.set_formula(col,'['+str(filename_)+']Sheet1!S')
        S2.set_formula(col,'['+str(filename_)+']Sheet1!T')
        S3.set_formula(col,'['+str(filename_)+']Sheet1!U')
        orientation_s.set_formula(col,'['+str(filename_)+']Sheet1!V')
        ellipticity_s.set_formula(col,'['+str(filename_)+']Sheet1!W')
        
        a.set_formula(col,'['+str(filename_)+']Sheet1!AB')
        b.set_formula(col,'['+str(filename_)+']Sheet1!AC')        
        orientation_f.set_formula(col,'['+str(filename_)+']Sheet1!AD')
        ellipticity_f.set_formula(col,'['+str(filename_)+']Sheet1!AE')
        
        op.wait('r', 0.0)
        
def closest(lst, K):
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

def row2column(wbname_from2Copy, freq_array, totalcolumn,wbname_from2Paste, Start_Column2Paste, Col_lname):
    
    Column2Paste = Start_Column2Paste
    for ii in range(len(freq_array)):
        
        string = []
        
        wks = op.find_sheet('w', '['+wbname_from2Copy+']Sheet1')
        freq = wks.to_list(0)
        #Finding Row index of nearest Frequency
        rownumber = freq.index(closest(freq,freq_array[ii]))
            
        #string.append("=[" + wbname_from2Copy + "]Sheet1!"+getColumnName(1)+str(rownumber))
        #string.append(0)
        
        #row to column for loop
        for i in range(totalcolumn):
            string.append("=[" + wbname_from2Copy + "]Sheet1!"+getColumnName(i+2)+str(rownumber+1))
        

        comment = wks.to_list(0)[rownumber]

        wks = op.find_sheet('w', '['+wbname_from2Paste+']Sheet1')
        wks.from_list(Column2Paste, string)
        wks.set_label(Column2Paste,Col_lname,'L')
        #wks.set_label(Column,'radians','U')
        wks.set_label(Column2Paste,str(comment),'C')
        
        Column2Paste = Column2Paste+1

def originexit():
    # Exit running instance of Origin.
    # Required for external Python but don't use with embedded Python.
    if op.oext:
        op.exit()
        
def originopen():
    # Set Origin instance visibility.
    # Important for only external Python.
    # Should not be used with embedded Python. 
    if op.oext:
        op.set_show(True)

def set_comment_freq(wks):
    wks.set_label(0,'Frequency','L')
    wks.set_label(0,'THz','U')
    
def set_comment_delay(wks):
    delay.set_label(0,'Delay','L')
    delay.set_label(0,'ps','U')
    
def datacropping(freq_point, freq, data):
    #Freq_point is the end frequency data point and freq is the frequency array
    rownumber = freq.index(closest(freq,freq_point))
    return freq[:rownumber+1], data[:rownumber+1] 


#Polarisation Fitting Function
def objectivefunc(phi, a, b, delta):
    phi = radians(phi)
    return abs(cos(phi))*sqrt((a*cos(phi)+b*sin(phi)*cos(delta))**2 + (b*sin(phi)*sin(delta))**2)
def savefitparams2worksheet(freq, a, b, delta, a_error, b_error, delta_error):
    import numpy as np
    wks = op.new_sheet(lname='FitParamters')

    wks.from_list(0,freq)
    wks.set_label(0,'Frequency','L')
    wks.set_label(0,'THz','U')

    wks.from_list(1,a)
    wks.set_label(1,'a','L')

    wks.from_list(2,b)
    wks.set_label(2,'b','L')

    wks.from_list(3,np.array(delta)*180/np.pi)
    wks.set_label(3,'Ellipticity','L')
    wks.set_label(3,'Degrees','U')

    wks.from_list(4,np.arctan(np.array(b)/np.array(a))*180/np.pi)
    wks.set_label(4,'Orientation Angle','L')
    wks.set_label(4,'Degrees','U')
    
    wks.from_list(5,a_error)
    wks.set_label(5,'Standard Deviation','L')
    wks.set_label(5,'Δa','C')

    wks.from_list(6,b_error)
    wks.set_label(6,'Standard Deviation','L')
    wks.set_label(6,'Δb','C')

    wks.from_list(7,delta_error)
    wks.set_label(7,'Standard Deviation','L')
    wks.set_label(7,'Δδ','C')
def parameter_separator(fit_data):
    import numpy as np
    fit_data = np.array(fit)
    return fit_data[:,0], fit_data[:,1], fit_data[:,2]

#Buttons 
class SnaptoCursor(object):
    def __init__(self, ax, x, y):
        self.ax = ax
        self.lx = ax.axhline(color='k',lw=0.5)  # the horiz line
        self.ly = ax.axvline(color='k')         # the vert line
        self.x = x
        self.y = y
        
        # text location in axes coords
        self.txt = ax.text(0.7, 0.1, '', transform=ax.transAxes)
    def snap(self, event):
        if not event.inaxes:
            return
        print(index_temp)
        x, y = event.xdata, event.ydata
        indx = min(np.searchsorted(self.x, x), len(self.x) - 1)
        index_temp.append(indx)
    
        x = self.x[indx]
        y = self.y[indx]
        
        # update the line positions
        self.lx.set_ydata(y)
        self.ly.set_xdata(x)
        self.txt.set_text('x=%1.2f, y=%1.2f' % (x, y))
        self.ax.figure.canvas.draw()
def add2pi(arg):
    global add2pi
    add2pi=index_temp[-1]
    new_y = add(add2pi, temp)
    plot.set_ydata(new_y)
    print('Add Index: ',add2pi)
def sub2pi(arg):
    global sub2pi
    sub2pi=index_temp[-1]
    new_y = sub(sub2pi,temp)
    plot.set_ydata(new_y)
    print('Sub Index: ',sub2pi)
def add(index, temp):
    new_y = temp;
    new_y[index:] = new_y[index:]+360
    temp = new_y
    return new_y
def sub(index, temp):
    new_y = temp;
    new_y[index:] = new_y[index:]-360
    temp = new_y
    return new_y

In [ ]:
#OpenOrigin2021
# Very useful, especially during development, when you are
# liable to have a few uncaught exceptions.
# Ensures that the Origin instance gets shut down properly.
# Note: only applicable to external Python.
import sys
def origin_shutdown_exception_hook(exctype, value, traceback):
    '''Ensures Origin gets shut down if an uncaught exception'''
    op.exit()
    sys.__excepthook__(exctype, value, traceback)
if op and op.oext:
    sys.excepthook = origin_shutdown_exception_hook
    
originopen()

In [ ]:
#Generates filename and save as text files
filepath = filename_generator(date, run, total_files,startpos,'FT')
np.savetxt('analysisfilename.csv',filepath,delimiter=',',fmt='%s')

In [ ]:
#df = pd.read_csv('analysisfilename.txt')
df = pd.read_csv('analysisfilename.csv',header=None).to_numpy()
filepath = df[:,0].tolist()
comment  = df[:,1].tolist()

In [ ]:
#Import Files
importfiles(filepath)
#importfiles_cut(filepath,7300)

In [ ]:
#Extract Amplitude, Phase and Delay
amplitude = op.new_sheet(lname='Amplitude')
collector('Q',amplitude,filepath,comment,'Amplitude','a.u.')
amplitude.set_formula('A','['+filepath[0]+']Sheet1!L')
set_comment_freq(amplitude)

phase = op.new_sheet(lname='Phase')
collector('R',phase,filepath,comment,'Phase','Degree')
set_comment_freq(phase)

delay = op.new_sheet(lname='Delay')
collector('K',delay,filepath,comment,'Amplitude','V')
delay.set_formula('A','['+filepath[0]+']Sheet1!J')
set_comment_delay(delay)

In [ ]:
#Extract Transmittance
transmittance = op.new_sheet(lname='Transmittance')
collector_trans_phase(transmittance,'AmplitudeAverage','B','/')
set_comment_freq(transmittance)

In [ ]:
originexit()

In [ ]:
#Column Name for Phase Correction
col = 2

In [ ]:
#Phase Correction loop
wks = op.find_sheet('w','Phase')
print(wks)
#freq = wks.to_list(0)
#data = wks.to_list(getColumnName(col+1))

#datacropping(3THz, Frequency column, Phase Column)
freq, data = datacropping(3,wks.to_list(0),wks.to_list(getColumnName(col+1)))

index_temp = []   #Temporary Index values of start and stop button
temp=[]
temp = np.array(data)

#Plots Reference and Sample delay plot
plt.close()
plt.close()
fig, ax = plt.subplots()
#ax.set_ylim(-200,4)
ax.set_xlim(0,3)
ax.set_xlabel('Frequency (THz)')
ax.set_ylabel('Phase')
ax.set_title('Phase Correction')
plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
plot, = ax.plot(freq, data, '-',label='Reference',marker='o')
plt.legend(loc='upper right')
#ax.plot(xs, ys, '-',label='Sample',markersize=5)'''

#Add 2*pi button plot Buttons
button_add = widgets.Button(description = 'Add')   
button_add.on_click(add2pi)
display(button_add)

#Sub 2*pi button plot Buttons
button_sub = widgets.Button(description = 'Sub')   
button_sub.on_click(sub2pi)
display(button_sub)

#snapping cursor to Reference
snap_cursor = SnaptoCursor(ax, freq, data)
fig.canvas.mpl_connect('button_press_event', snap_cursor.snap)
plt.legend(loc='upper right')
plt.show()


In [ ]:
#Paste phase corrected loop
datatopaste = temp
wks2 = op.find_sheet('w','Phase1')
wks2.from_list(getColumnName(col+1),datatopaste)